# QS Solve

In [1]:
from huggingface_hub import login

login()

In [2]:
from distutils.dir_util import copy_tree
from huggingface_hub import Repository, snapshot_download, create_repo, get_full_repo_name


def copy_repository_template():
    # Clone the repo and extract the local path
    template_repo_id = "lewtun/distilbert-base-uncased-finetuned-squad-d5716d28"
    commit_hash = "be3eaffc28669d7932492681cd5f3e8905e358b4"
    template_repo_dir = snapshot_download(template_repo_id, revision=commit_hash)
    # Create an empty repo on the Hub
    model_name = template_repo_id.split("/")[1]
    create_repo(model_name, exist_ok=True)
    # Clone the empty repo
    new_repo_id = get_full_repo_name(model_name)
    new_repo_dir = model_name
    repo = Repository(local_dir=new_repo_dir, clone_from=new_repo_id)
    # Copy files
    copy_tree(template_repo_dir, new_repo_dir)
    # Push to Hub
    repo.push_to_hub()

In [3]:
copy_repository_template()

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/root/hf-learning/202509_chapter4/distilbert-base-uncased-finetuned-squad-d5716d28 is already a clone of https://huggingface.co/yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28. Make sure you pull the latest changes with `repo.git_pull()`.


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/2.05k [00:00<?, ?B/s]

EOF
error: failed to push some refs to 'https://huggingface.co/yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28'



OSError: EOF
error: failed to push some refs to 'https://huggingface.co/yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28'


In [6]:
from transformers import pipeline

model_checkpoint = get_full_repo_name("distillbert-base-uncased-finetuned-squad-d5716d28")
reader = pipeline("question-answering", model=model_checkpoint)

/root/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


OSError: yiwenX/distillbert-base-uncased-finetuned-squad-d5716d28 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [7]:
model_checkpoint = get_full_repo_name("distilbert-base-uncased-finetuned-squad-d5716d28")
reader = pipeline("question-answering", model=model_checkpoint)

ValueError: Could not load model yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForQuestionAnswering'>,). See the original errors:

while loading with AutoModelForQuestionAnswering, an error is thrown:
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.12/site-packages/transformers/pipelines/base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/site-packages/transformers/modeling_utils.py", line 288, in _wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/

In [8]:
from huggingface_hub import list_repo_files

list_repo_files(repo_id=model_checkpoint)

['.gitattributes']

In [9]:
from transformers import AutoConfig

pretrained_checkpoint = "distilbert-base-uncased"
config = AutoConfig.from_pretrained(pretrained_checkpoint)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [10]:
config.push_to_hub(model_checkpoint, commit_message="Add config.json")

CommitInfo(commit_url='https://huggingface.co/yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28/commit/859a756ce7fcd2dbff04283f89607c5571362c60', commit_message='Add config.json', commit_description='', oid='859a756ce7fcd2dbff04283f89607c5571362c60', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28', endpoint='https://huggingface.co', repo_type='model', repo_id='yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28'), pr_revision=None, pr_num=None)

In [11]:
reader = pipeline("question-answering", model=model_checkpoint, revision="main")

context = r"""
Extractive Question Answering is the task of extracting an answer from a text
given a question. An example of a question answering dataset is the SQuAD
dataset, which is entirely based on that task. If you would like to fine-tune a
model on a SQuAD task, you may leverage the
examples/pytorch/question-answering/run_squad.py script.

🤗 Transformers is interoperable with the PyTorch, TensorFlow, and JAX
frameworks, so you can use your favourite tools for a wide variety of tasks!
"""

question = "What is extractive question answering?"
reader(question=question, context=context)

config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

ValueError: Could not load model yiwenX/distilbert-base-uncased-finetuned-squad-d5716d28 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForQuestionAnswering'>, <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM'>). See the original errors:

while loading with AutoModelForQuestionAnswering, an error is thrown:
Traceback (most recent call last):
  File "/root/miniconda3/lib/python3.12/site-packages/transformers/pipelines/base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/lib/python3.12/site-packages/transformers/modeling_utils.py", line 288, in _wrapper
    return func(*args, 